In [1]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as f
from pyspark.sql.types import *

In [2]:
! pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_312
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
data_path = '/media/daniel/Seagate Basic/spark_data/libros/' # en esta carpeta deben encontrarse los 4 ficheros

In [4]:
spark = SparkSession.builder.master("local[*]")\
          .appName("practica 2 Ecosistema Spark parte 3")\
          .getOrCreate()

22/01/09 12:47:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# 1. Carga de dataframes

El primer paso consiste en cargar los tres dataframes. Para ello, se leen los ficheros csv con la API de Spark.

In [26]:
ratings = spark.read.csv(data_path+'BX-Book-Ratings.csv', sep=';', header=True, inferSchema=True, encoding='latin1')
ratings.printSchema()

root
 |-- User-ID: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: integer (nullable = true)



In [27]:
books = spark.read.csv(data_path+'BX-Books.csv', sep=';', header=True, inferSchema=True, encoding='latin1')
books.printSchema()

root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Image-URL-S: string (nullable = true)
 |-- Image-URL-M: string (nullable = true)
 |-- Image-URL-L: string (nullable = true)



In [28]:
users = spark.read.csv(data_path+'BX-Users.csv', sep=';', header=True, inferSchema=True, encoding='latin1')
users.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)



# 2. Consultas

### a. Lista de usuarios junto con el número de libros que han valorado

Se agrupa por usuario y se cuenta el número de reseñas que estan registradas en el fichero de reseñas.

In [13]:
ratings_count = ratings.groupBy('User-ID').count()

In [21]:
ratings_count.sort(f.col("count").desc()).show(20)

+-------+-----+
|User-ID|count|
+-------+-----+
|  11676|13602|
| 198711| 7550|
| 153662| 6109|
|  98391| 5891|
|  35859| 5850|
| 212898| 4785|
| 278418| 4533|
|  76352| 3367|
| 110973| 3100|
| 235105| 3067|
| 230522| 2991|
|  16795| 2948|
| 234623| 2674|
|  36836| 2529|
|  52584| 2512|
| 245963| 2507|
| 204864| 2504|
|  55492| 2459|
| 185233| 2448|
| 171118| 2421|
+-------+-----+
only showing top 20 rows



### b. Rating máximo recibido por cada editorial


En primer lugar se cruzan los datos de reseñas y de libros mediante un join sobre la clave ISBN. Despues se busca el máximo de las puntuaciones agregando por editorial.

In [8]:
joined = books.join(ratings, on='ISBN', how='left')

In [ ]:
max_rating = joined.select('Publisher', 'Book-Rating').groupBy('Publisher').max().sort(f.col("max(Book-Rating)").desc())

max_rating.show()

+--------------------+----------------+
|           Publisher|max(Book-Rating)|
+--------------------+----------------+
|Heinemann Educati...|              10|
|            Sterling|              10|
|   Editions du Seuil|              10|
|Abdo &amp; Daught...|              10|
|     South End Press|              10|
|    Gooseberry Patch|              10|
|    Thomas Nelson(J)|              10|
|Harpercollins Canada|              10|
| Harri Deutsch, Ffm.|              10|
|      Fog City Press|              10|
|       Ogden Pub Inc|              10|
|     Barricade Books|              10|
|      Wrox Press Inc|              10|
|Clothespin Fever ...|              10|
|     Scholastic, Inc|              10|
| Indigo Publications|              10|
|Quadrillion Publi...|              10|
|  Firefly Publishing|              10|
|Nettle Creek Publ...|              10|
|           Jugglebug|              10|
+--------------------+----------------+
only showing top 20 rows



### c. Nombre del autor que ha recibido más ratings

En este caso, reutilizamos el dataframe concatenado ("joined") del ejercicio anterior. Al haber hecho un left join, cada autor aparece tantas veces como haya aparecido el ISBN de su libro en el fichero de reseñas. Por tanto, basta hacer un recuento de cuántas veces aparece un autor en el dataframe "joined".

In [ ]:
count_author = joined.select('Book-Author')\
                        .groupBy('Book-Author')\
                        .count()\
                        .select('Book-Author', f.col('count')\
                        .alias('ratings_count'))\
                        .sort(f.col("ratings_count").desc())

count_author.show(20)

+--------------------+-------------+
|         Book-Author|ratings_count|
+--------------------+-------------+
|        Stephen King|        10055|
|        Nora Roberts|         8429|
|        John Grisham|         6010|
|     James Patterson|         5845|
|  Mary Higgins Clark|         4778|
|      Dean R. Koontz|         4313|
|          Tom Clancy|         4036|
|      Danielle Steel|         3726|
|         Sue Grafton|         3457|
|     Janet Evanovich|         3350|
|           Anne Rice|         3030|
|    Michael Crichton|         2921|
|       J. K. Rowling|         2909|
|        JOHN GRISHAM|         2808|
|        V.C. Andrews|         2785|
|        Sandra Brown|         2663|
|     Nicholas Sparks|         2650|
|         R. L. Stine|         2606|
|        Rich Shapero|         2502|
|Patricia Daniels ...|         2461|
+--------------------+-------------+
only showing top 20 rows



# 3. Window Functions

### a. ¿Cuál es el título del libro con mayor número de ratings para cada editorial?


Para este caso se usa una window function tal que:

- Particionamiento: Editorial
- Ordenamiento: Recuento de reseñas (descendente)

Despues, se calcula el número de reseñas para cada libro y editorial. Luego, se particiona por editorial y ordena por número de reseñas. Se escogeran las primeras filas en el rankeo para dar el resultado pedido.

In [29]:
joined_count = joined.select('Publisher', 'Book-Title')\
                        .groupBy('Publisher', 'Book-Title')\
                        .count().select('Publisher', 'Book-Title', f.col('count').alias('ratings_count')) # calculo del numero de reseñas

window = Window.partitionBy(joined_count.Publisher).orderBy(joined_count.ratings_count.desc())

Primero se calcula el número de reseñas para cada libro y editorial. Despues se particiona por editorial y ordena por número de reseñas. Se escogeran las primeras filas en el rankeo para dar el resultado pedido.

In [30]:
result_window = joined_count.withColumn('rank', f.dense_rank().over(window)) # calculo del rankeo

result_filter_by_window = result_window.where('rank == 1')  # resultado final

Se puede apreciar que se calcula una clasificación (rank) en función del número de reseñas a nivel editorial de cada libro. En el siguiente ejemplo se observan los valores de ratings_count y rank de las filas de Adams Media Corp. Se ve que aaprecen ordenadas por el numero de reseñas y que si existen duplicados en el numero de reseñas, estas se rankean en la misma posicion.

In [17]:
result_window.show(10)

+-------------------+--------------------+-------------+----+
|          Publisher|          Book-Title|ratings_count|rank|
+-------------------+--------------------+-------------+----+
|A. &amp; M. Muchnik|Comuna Verdad (An...|            1|   1|
|   Adams Media Corp|365 Tv-Free Activ...|           10|   1|
|   Adams Media Corp|The Verbally Abus...|            7|   2|
|   Adams Media Corp|Show Biz Tricks f...|            4|   3|
|   Adams Media Corp|Jacob Marley's Ch...|            4|   3|
|   Adams Media Corp|The Complete Sing...|            3|   4|
|   Adams Media Corp| Mr. Cheap's Chicago|            2|   5|
|   Adams Media Corp|The Everything We...|            2|   5|
|   Adams Media Corp|Museum of Science...|            2|   5|
|   Adams Media Corp|Knock 'Em Dead: T...|            2|   5|
+-------------------+--------------------+-------------+----+
only showing top 10 rows



Y tambien se observa que para obtener el libro con mas reseñas de cada editorial basta filtrar por rank = 1 (continuando con el ejemplo anterior, observar la segunda fila del siguiente resultado)

In [20]:
result_filter_by_window.sort(f.col("ratings_count").desc()).show()

+--------------------+--------------------+-------------+----+
|           Publisher|          Book-Title|ratings_count|rank|
+--------------------+--------------------+-------------+----+
|             Too Far|         Wild Animus|         2502|   1|
|       Little, Brown|The Lovely Bones:...|         1295|   1|
|           Doubleday|   The Da Vinci Code|          884|   1|
|           Perennial|Divine Secrets of...|          732|   1|
|         Picador USA|The Red Tent (Bes...|          723|   1|
|Dell Publishing C...|     A Painted House|          649|   1|
|        Warner Books|        The Notebook|          647|   1|
|   Vintage Books USA|Snow Falling on C...|          618|   1|
|       Penguin Books|The Secret Life o...|          615|   1|
|         Pocket Star| Angels &amp; Demons|          586|   1|
|Arthur A. Levine ...|Harry Potter and ...|          575|   1|
|      Back Bay Books|The Pilot's Wife ...|          568|   1|
|          Scholastic|Harry Potter and ...|          55

### b. ¿Cuál es la diferencia entre el número de ratings de cada libro y el número de ratings del libro con mayor número de ratings de la misma editorial?

Para este caso, se calcula el máximo de ratings con la misma partición que en el ajercicio anterior. Esta operación asignará a cada fila de la partición el rating máximo dentro de la partición. Así, se vuelve sencillo calcular dicha diferencia, ya que el problema se reduce a operar dos columnas del dataframe. 

In [31]:
ratings_diff = joined_count.withColumn('max_ratings_diff',
                                       f.max(joined_count.ratings_count).over(window) - joined_count.ratings_count)
                                        # maximo de la particion - rating de la fila

A continuación se observan algunos ejemplos del resultado. El máximo de la partición tiene diferencia 0 y el resto valores estrictamente mayores a 1. Si se pone el foco sobre Adams Media Corp, se ve que el máximo tiene valor 0 (10 reseñas) y el resto se ordenan de menor a mayor según su diferencia con respecto al máximo (10).

In [35]:
ratings_diff.sort(f.col("Publisher").asc(), f.col("ratings_count").desc()).show()

+--------------------+--------------------+-------------+----------------+
|           Publisher|          Book-Title|ratings_count|max_ratings_diff|
+--------------------+--------------------+-------------+----------------+
| Editions P. Terrail|Afrique noire: Ma...|            1|               0|
|     Tri-State Press|Fluoridation: A m...|            1|               0|
|           "Corvina"|Jan Matejko (W kr...|            1|               0|
|             "Nauka"|Tabakokurenie i m...|            2|               0|
|  "Otokar Kersovani"|Gdje pijevac ne p...|            1|               0|
|             'K' Pub|Trilogi Khadijah ...|            1|               0|
|(3 Queen Sq., WC1...|      The big chapel|            1|               0|
|(49 Poland St., W...|Japan: the years ...|            1|               0|
|               10-18|Le roman de Trist...|            8|               0|
|               10-18|Le Chat qui sniff...|            6|               2|
|               10-18|   

# 4. Conclusión

A modo de reflexión final, se observa que la API de Spark permite realizar consultas de manera cómoda e intuitiva. Por otro lado, tiene limitaciones en cuánto a consultas agregadas. Éstas se pueden subsanar de manera eficiente con el uso de Window Functions, las cuales simplifican el agregado de información tanto desde el punto de vista de la síntaxis como en eficiencia de computación.